Importing necessary files

In [ ]:
import os
import zipfile
import random
import json
import pandas as pd
from shutil import copyfile


Downloading dataset

In [ ]:
!wget https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/6312cfc2-6930-4530-9589-105499b29d5e -O images.zip

--2022-06-25 00:49:09--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/6312cfc2-6930-4530-9589-105499b29d5e
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.97.171
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.97.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6758950318 (6.3G) [application/x-zip-compressed]
Saving to: ‘images.zip’

images.zip          100%[===================>]   6.29G  29.4MB/s    in 3m 42s  

2022-06-25 00:52:51 (29.0 MB/s) - ‘images.zip’ saved [6758950318/6758950318]



Unziping Dataset

In [ ]:
local_zip = '/content/images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Temp')

Downloading annotations

In [ ]:
!wget https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/9b3e789a-5a5c-48b3-8a2c-2c91e9307c2a/file_downloaded -O'/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Temp/annotations.json'

--2022-06-25 01:03:13--  https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/9b3e789a-5a5c-48b3-8a2c-2c91e9307c2a/file_downloaded
Resolving data.mendeley.com (data.mendeley.com)... 162.159.133.86, 162.159.130.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.133.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/0acef4de-4a98-4efb-bc8c-8c766e0a3ca0 [following]
--2022-06-25 01:03:14--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/0acef4de-4a98-4efb-bc8c-8c766e0a3ca0
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.92.96
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.92.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2649168 (2.5M) [application/j

Splitting Train and Validation files

In [ ]:
SrcDir = '/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Temp/images/'
ParentFile = '/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Data'

In [ ]:
TrainDir = os.path.join(ParentFile, 'Train/')
ValDir = os.path.join(ParentFile, 'Val/')
dfTrain = pd.DataFrame()
dfVal = pd.DataFrame()
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Temp/annotations.json', orient='records')

ImageNames = []
for FileName in os.listdir(SrcDir):

    ImagePath = (SrcDir + FileName )
    if os.path.getsize(ImagePath) > 0:
      ImageNames.append(FileName)
    else:
      print(FileName  + " is zero length, so ignoring.")

random.shuffle(ImageNames)
RandImageNames = ImageNames

SplitRatio = .75

TrainingSet = RandImageNames[: int(len(RandImageNames) * SplitRatio)]
ValSet = RandImageNames[int(len(RandImageNames) * SplitRatio):]

for TrainImage in TrainingSet:
  copyfile( SrcDir + TrainImage , TrainDir + TrainImage  )
  dfTrain = dfTrain.append(df[df["filename"] == TrainImage])

for ValImage in ValSet:
  copyfile( SrcDir + ValImage , ValDir + ValImage  )
  dfVal = dfVal.append(df[df["filename"] == ValImage])

#saving the train and validation images in separate files
TrainAnnot = dfTrain.to_json('/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Data/TrainAnnotations.json', orient='records', force_ascii=False)
ValAnnot = dfVal.to_json('/content/drive/MyDrive/Colab Notebooks/BanglaImageCaption/Data/ValAnnotations.json', orient='records', force_ascii=False)
  
